# The Brief: 


a predictive model using a Machine Learning algorithm to accurately predict the sales revenue for a given item in a specific store at a given date.

a forecasting model using a time-series analysis algorithm that will forecast the total sales revenue across all stores and items for the next 7 days

# 1. Data collection

In [1]:
#import libraries
import pandas as pd
import missingno as msno
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from joblib import dump
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

import re

In [2]:
#[1-2]Load the data into a DataFrame
sales_train = pd.read_csv('../data/raw/sales_train.csv',low_memory=False)
sales_test = pd.read_csv('../data/raw/sales_test.csv',low_memory=False)
calender_events= pd.read_csv('../data/raw/calendar_events.csv',low_memory=False)
calender= pd.read_csv('../data/raw/calendar.csv',low_memory=False)
items_weekly_sell_prices= pd.read_csv('../data/raw/items_weekly_sell_prices.csv',low_memory=False)

# 2. Explore dataset


# sales_train

In [3]:
# Display the 5 first rows of df Sales_train 
sales_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3


In [4]:
sales_train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1532', 'd_1533', 'd_1534', 'd_1535', 'd_1536', 'd_1537', 'd_1538',
       'd_1539', 'd_1540', 'd_1541'],
      dtype='object', length=1547)

In [5]:
random_rows_sample = sales_train[['dept_id', 'cat_id', 'store_id']].sample(n=5)

# Print the randomly selected rows with only columns 'A', 'B', and 'C'
print(random_rows_sample)

           dept_id     cat_id store_id
29806      FOODS_3      FOODS     WI_3
22062  HOUSEHOLD_1  HOUSEHOLD     WI_1
22958      FOODS_1      FOODS     WI_1
28569  HOUSEHOLD_2  HOUSEHOLD     WI_3
27704    HOBBIES_1    HOBBIES     WI_3


In [6]:
sales_train.shape


(30490, 1547)

In [7]:
#unique values of cat_id
unique_cat_id = sales_train['cat_id'].unique()
print(unique_cat_id)

['HOBBIES' 'HOUSEHOLD' 'FOODS']


In [8]:
#unique values of store_id
unique_store_id = sales_train['store_id'].unique()
print(unique_store_id)

['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']


In [9]:
#unique values of dept_id
unique_dept_id = sales_train['dept_id'].unique()
print(unique_dept_id)

['HOBBIES_1' 'HOBBIES_2' 'HOUSEHOLD_1' 'HOUSEHOLD_2' 'FOODS_1' 'FOODS_2'
 'FOODS_3']


In [10]:
#unique values of item_id	
unique_item_id = sales_train['item_id'].unique()
print(unique_item_id)

['HOBBIES_1_001' 'HOBBIES_1_002' 'HOBBIES_1_003' ... 'FOODS_3_825'
 'FOODS_3_826' 'FOODS_3_827']


In [11]:
unique_item_count = sales_train['item_id'].nunique()

# Print the count of unique item IDs
print("Count of unique item IDs:", unique_item_count)

Count of unique item IDs: 3049


# calender events

In [12]:
# Display the 5 first rows of df calender_events
calender_events.head()

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious


In [13]:
# Display the dimentions (shape) of df
calender_events.shape

(167, 3)

In [14]:
#unique values of event_name
unique_events = calender_events['event_name'].unique()
print(unique_events)

['SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart' 'LentWeek2'
 'StPatricksDay' 'Purim End' 'Easter' 'OrthodoxEaster' 'Pesach End'
 'Cinco De Mayo' "Mother's day" 'MemorialDay' 'NBAFinalsStart'
 'NBAFinalsEnd' "Father's day" 'IndependenceDay' 'Ramadan starts'
 'Eid al-Fitr' 'LaborDay' 'ColumbusDay' 'Halloween' 'EidAlAdha'
 'VeteransDay' 'Thanksgiving' 'Christmas' 'Chanukah End' 'NewYear'
 'OrthodoxChristmas' 'MartinLutherKingDay']


In [15]:
#unique values of event_type
unique_event_type = calender_events['event_type'].unique()
print(unique_event_type)

['Sporting' 'Cultural' 'National' 'Religious']


In [16]:
# Convert the date column to a datetime format
calender_events['date'] = pd.to_datetime(calender_events['date'])

# Extract unique years from the date column
unique_years = calender_events['date'].dt.year.unique()

# Print the unique years
print(unique_years)

[2011 2012 2013 2014 2015 2016]


# calender

In [17]:
# Display the 5 first rows of df calender
calender.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


In [18]:
#Display tail of calender
calender.tail()

,date,wm_yr_wk,d
1964,2016-06-15,11620,d_1965
1965,2016-06-16,11620,d_1966
1966,2016-06-17,11620,d_1967
1967,2016-06-18,11621,d_1968
1968,2016-06-19,11621,d_1969


In [19]:
calender.shape

(1969, 3)

items_weekly_sell_prices

In [20]:
# Display the 5 first rows of df items_weekly_sell_prices
items_weekly_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [21]:
items_weekly_sell_prices.columns

Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')

In [22]:
#Dilsplay tail of items_weekly_sell_prices
items_weekly_sell_prices.tail()

,store_id,item_id,wm_yr_wk,sell_price
6841116,WI_3,FOODS_3_827,11617,1.0
6841117,WI_3,FOODS_3_827,11618,1.0
6841118,WI_3,FOODS_3_827,11619,1.0
6841119,WI_3,FOODS_3_827,11620,1.0
6841120,WI_3,FOODS_3_827,11621,1.0


In [23]:
#dimension
items_weekly_sell_prices.shape

(6841121, 4)

# Merging Tables (train)

merge sale_train with calender

In [24]:
import pandas as pd


# Select columns starting with 'd_' in sales_train for merging
columns_to_merge = [col for col in sales_train.columns if col.startswith('d_')]

# Melt the sales_train DataFrame to have a 'day' column for day values
melted_sales_train = pd.melt(sales_train, id_vars=['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], value_vars=columns_to_merge, var_name='day', value_name='count')

# Merge the melted sales_train DataFrame with the calendar DataFrame based on the 'day' column
result_df = pd.merge(melted_sales_train, calender[['d', 'date', 'wm_yr_wk']], left_on='day', right_on='d', how='left')

# Drop the 'd' and 'day' columns as they are no longer needed
result_df = result_df.drop([ 'd'], axis=1)




In [25]:
result_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,day,count,date,wm_yr_wk
46985085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1541,2,2015-04-18,11512
46985086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512
46985087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512
46985088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512
46985089,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1541,3,2015-04-18,11512


In [26]:
result_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day',
       'count', 'date', 'wm_yr_wk'],
      dtype='object')

In [27]:
import pandas as pd


# Convert the 'date' column to datetime type in both DataFrames
result_df['date'] = pd.to_datetime(result_df['date'])

# Merge the two DataFrames using a left join and fill missing values with 0
result_df2 = pd.merge(result_df, calender_events, on='date', how='left').fillna({'event_name': '0', 'event_type': '0'})

# Display the resulting DataFrame
print(result_df2)


                                     id        item_id    dept_id   cat_id   
0         HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES  \
1         HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47107045    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47107046    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47107047    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47107048    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47107049    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id     day  count       date  wm_yr_wk 

In [28]:
result_df2.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,count,date,wm_yr_wk,event_name,event_type
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,0,0


In [29]:
import pandas as pd


# Merge the two DataFrames using a left join
merged_df = pd.merge(result_df2, items_weekly_sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

# Display the resulting DataFrame
print(merged_df)


                                     id        item_id    dept_id   cat_id   
0         HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES  \
1         HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2         HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3         HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4         HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47107045    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47107046    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47107047    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47107048    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47107049    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id     day  count       date  wm_yr_wk 

In [30]:
# Add a new 'revenue' column by multiplying 'count' and 'sell_price'
merged_df['revenue'] = merged_df['count'] * merged_df['sell_price']

In [31]:
merged_df.sample(5)

,id,item_id,dept_id,cat_id,store_id,state_id,day,count,date,wm_yr_wk,event_name,event_type,sell_price,revenue
3124710,FOODS_3_310_TX_1_evaluation,FOODS_3_310,FOODS_3,FOODS,TX_1,TX,d_102,0,2011-05-10,11115,0,0,NaN,NaN
26949037,FOODS_2_149_WI_2_evaluation,FOODS_2_149,FOODS_2,FOODS,WI_2,WI,d_882,1,2013-06-28,11322,0,0,7.47,7.47
37717275,HOUSEHOLD_1_008_TX_2_evaluation,HOUSEHOLD_1_008,HOUSEHOLD_1,HOUSEHOLD,TX_2,TX,d_1234,0,2014-06-15,11420,NBAFinalsEnd,Sporting,5.47,0.00
6857303,HOBBIES_1_107_WI_3_evaluation,HOBBIES_1_107,HOBBIES_1,HOBBIES,WI_3,WI,d_224,0,2011-09-09,11132,0,0,3.98,0.00
33452298,FOODS_1_110_CA_2_evaluation,FOODS_1_110,FOODS_1,FOODS,CA_2,CA,d_1096,0,2014-01-28,11353,0,0,1.50,0.00


getting categorical columns

In [32]:
#add year, month and day

merged_df["year"] = merged_df["date"].dt.year
merged_df["month"] = merged_df["date"].dt.month
merged_df["day"] = merged_df["date"].dt.day


In [33]:
merged_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,count,date,wm_yr_wk,event_name,event_type,sell_price,revenue,year,month
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,29,0,2011-01-29,11101,0,0,NaN,NaN,2011,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,29,0,2011-01-29,11101,0,0,NaN,NaN,2011,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,29,0,2011-01-29,11101,0,0,NaN,NaN,2011,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,29,0,2011-01-29,11101,0,0,NaN,NaN,2011,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,29,0,2011-01-29,11101,0,0,NaN,NaN,2011,1


In [34]:
#Save the sets into the folder `data/processed`
# Solution

merged_df.to_csv('../data/processed/merged_df.csv', index=False)
